# **Actividad 3 - modelos supervizados y no supervizados**

José Ricardo Munguía Marín
A01795660@tec.mx


## 1 Introducción Teórica

### Aprendizaje supervisado vs no supervisado

El aprendizaje supervisado es un enfoque de machine learning donde el modelo se entrena con datos etiquetados, es decir, con ejemplos cuya respuesta correcta ya se conoce. Muy parecido al concepto de Regresiones Lineales.


El objetivo es que el modelo aprenda la relación entre las características de entrada y el resultado, para luego poder predecir correctamente las etiquetas de nuevos datos.

Un caso típico de aprendizaje supervisado con aplicación a nuestro proyecto es los de tipo clasificación, donde se entrena un modelo para distinguir categorías (por ejemplo, reseñas positivas vs reseñas negativas), utilizando ejemplos previamente categorizados como positivos o negativos.


En nuestro proyecto,contamos con reseñas de libros y sabemos si su calificación (score) indica una percepción positiva o negativa de la calidad; con esos datos entrenamos un modelo que pueda predecir el sentimiento de una reseña nueva.

Por otro lado, el aprendizaje no supervisado se utiliza cuando no disponemos de etiquetas o categorías predefinidas en los datos. En lugar de predecir un valor objetivo conocido, el modelo intenta descubrir patrones o estructuras ocultas por sí mismo.

Un ejemplo común que vamos a aplicar es el agrupamiento (clustering), donde el algoritmo organiza los datos en grupos basándose en su similitud, sin que nadie le haya dicho de antemano cuáles son esos grupos.


###Modelo DecisionTreeClassifier para clasificar reseñas

Para la tarea de clasificación de reseñas (aprendizaje supervisado), decidimos usar un árbol de decisión mediante la clase DecisionTreeClassifier de PySpark.

Un árbol de decisión es un modelo de clasificación que aprende a tomar decisiones basadas en una serie de preguntas sobre las características de los datos, organizadas en forma de árbol. Elegimos este modelo por varias razones:

1. Interpretabilidad: Los árboles de decisión son fáciles de interpretar gracias a que las predicciones se realizan siguiendo ramas lógicas if/then. Esto significa que podemos entender por qué el modelo clasificó una reseña como positiva o negativa al seguir las reglas del árbol. Esta claridad será muy útil para analizar resultados posteriormente cuando un producto de ML es entregado.

2. Manejo de variables heterogéneas: El DecisionTreeClassifier puede trabajar con características numéricas como categóricas sin necesidad de mucho procesamiento adicional.

3. Sencillez y eficacia en clasificación binaria: Para distinguir reseñas positivas vs negativas (por ejemplo, podríamos definir positiva si la calificación del usuario es ≥ 4 estrellas, caso contrario negativa).

En resumen, el DecisionTreeClassifier parece adecuado para empezar a experimentar pues provee un buen balance entre interpretabilidad y rendimiento para la clasificación de reseñas en positivas o negativas.


###Modelo KMeans para agrupar libros

Para la tarea de agrupamiento de libros según popularidad y antigüedad (aprendizaje no supervisado), utilizamos el algoritmo K-Means mediante la clase KMeans de PySpark.

La idea es pedirle al algoritmo que divida nuestros libros en K grupos distintos, donde K es un número que definimos (por ejemplo, 3, 5, 10, etc.), de forma que los libros dentro de cada grupo sean lo más similares posible entre sí en términos de variables.

Elegimos K-Means principalmente porque:
1. Simplicidad y eficiencia: K-Means es fácil de entender e implementar, y suele converger rápidamente. Esto lo hace práctico para explorar agrupaciones iniciales en un conjunto de datos grande. Encaja bien con nuestro dataset de reseñas de Amazon (que contiene millones de registros). Usando PySpark, podemos aprovechar K-Means para procesar muchos libros en paralelo sin demasiada complicación en el código.

- Resultados interpretables: da centroides para cada clúster, que básicamente representan el libro “promedio” en términos de popularidad y antigüedad en ese grupo.

-	No requiere etiqueta previa: Dado que no teníamos una variable objetiva para este análisis (no hay una clasificación preexistente de libros por categorías de popularidad/antigüedad), K-Means era una opción natural.

Vamos a usar K-Means porque queríamos explorar la estructura de la base de datos de libros en términos de popularidad y antigüedad.

## 2 Selección de Datos

Partimos de una muestra representativa de las reseñas de Amazon Book Reviews (llamada muestra M en el notebook).
Ahora, para poder craer esta muestra M, se crearon primero las particiones correspondientes para crearla. Estás particiones fueron creadas en la cartividad anterior, para asegurar un muestreo representativo basado en cada tipo de variable.


In [1]:
# -*- coding: utf-8 -*-

# IMPORTACIONES
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, when, udf
from pyspark.sql.types import StringType
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, MinMaxScaler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

In [2]:

# ----------------------------------------
# SESIÓN SPARK
# ----------------------------------------
spark = SparkSession.builder.appName("MLlibEquipo52").getOrCreate()

# ----------------------------------------
# PARTE 2: Selección de los datos
# CARGA Y UNIÓN DE DATOS
# ----------------------------------------
books = spark.read.csv("books_data.csv", header=True, inferSchema=True)
ratings = spark.read.csv("Books_rating.csv", header=True, inferSchema=True)

books = books.withColumnRenamed("title", "Title")
df = ratings.join(books.select("Title", "categories", "publishedDate", "ratingsCount"), on="Title", how="inner")

# FILTRADO DE NULOS PARA VARIABLES CLAVE
# ----------------------------------------
df_clean = df.filter(col("ratingsCount").isNotNull() & col("review/score").isNotNull())

# PARTICIONAMIENTO SEGÚN CRITERIOS DE LA ACTIVIDAD 2
# ----------------------------------------
df_clean = df_clean.withColumn("ratings_partition", when(col("ratingsCount") <= 1000, "baja").otherwise("alta"))
df_clean = df_clean.withColumn("score_partition", when(col("review/score") < 4.0, "baja").otherwise("alta"))

part_A = df_clean.filter((col("ratings_partition") == "baja") & (col("score_partition") == "baja"))
part_B = df_clean.filter((col("ratings_partition") == "baja") & (col("score_partition") == "alta"))
part_C = df_clean.filter((col("ratings_partition") == "alta") & (col("score_partition") == "baja"))
part_D = df_clean.filter((col("ratings_partition") == "alta") & (col("score_partition") == "alta"))

# MUESTREO POR PARTICIÓN
# ----------------------------------------
sample_A = part_A.sample(False, 0.3, seed=42) #Partición 1: muestreo Aleatorio Simple al 30%
sample_B = part_B.sample(False, 0.05, seed=42) #Partición 2: muestreo Aleatorio Simple al 5%

#Partición 3: muestreo sistemático, toma 1 de cada 5 registros mediante indexado con zipWithIndex() y filtro
sample_C = part_C.rdd.zipWithIndex().filter(lambda x: x[1] % 5 == 0).map(lambda x: x[0]).toDF()

#Partición 4: Muestra Completa
sample_D = part_D

muestra_M = sample_A.union(sample_B).union(sample_C).union(sample_D)


## 3 Preparción de Datos

Antes de modelar, realizamos limpieza básica. Esto incluyó eliminar filas con valores nulos o incompletos que pudieran afectar el entrenamiento (por ejemplo, libros sin fecha de publicación o reseñas sin puntuación).

También convertimos tipos de datos cuando fue necesario por ejemplo, asegurándonos de que el campo de número de calificaciones (ratingsCount) estuviera en formato numérico y no como texto.


####Transformación y feature engineering:

Esta es una etapa clave donde preparamos las variables de entrada para los modelos:
- **Codificación de categóricas**
- **Extracción de características numéricas**

Pero las más importantes fueron:

- **Binarización de la variable objetivo**: Para el modelo supervisado, definimos la etiqueta label indicando percepción positiva o negativa de la reseña. Se consideró positiva (label = 1) si la calificación de la reseña era ≥ 4.0 (asumiendo que las calificaciones son de 1 a 5 estrellas), y negativa (label = 0) si era menor a 4.0. De este modo convertimos el problema en una clasificación binaria simple. Esto fue más sugestivo, no quiere decir que los otros libros fueran malos.

- **Ensamblaje de features:** Con las características preparadas (por ejemplo: categoría codificada, año de publicación, número de reseñas, posiblemente otras como la puntuación promedio, etc.), las combinamos en una única columna de features utilizando VectorAssembler de PySpark. Esto nos da un vector numérico por cada libro/reseña, que representa todas las variables predictoras.

- Norma**lización (escalado)**: Para el algoritmo KMeans en particular, es recomendable normalizar las características numéricas, ya que este algoritmo basa sus agrupaciones en distancias. En nuestro caso, la escala de antigüedad (por año) y la de popularidad (por número de reseñas, que puede ir de decenas a miles) son muy diferentes.

In [3]:

# ----------------------------------------
# PARTE 3: Preparación de los datos
# TRANSFORMACIONES PREVIAS
# ----------------------------------------
muestra_M = muestra_M.withColumn("year", year(col("publishedDate")))

# LIMPIEZA Y CONVERSIÓN DE CATEGORIES A STRINGS SIMPLES
to_string_udf = udf(lambda x: str(x).strip("[]'").split(",")[0] if x else None, StringType())
muestra_M = muestra_M.withColumn("categories", to_string_udf("categories"))
muestra_M = muestra_M.filter(col("categories").isNotNull())

# CONVERSIÓN DE ratingsCount A TIPO NUMÉRICO
muestra_M = muestra_M.withColumn("ratingsCount", col("ratingsCount").cast("double"))

# BINARIZACIÓN DE LABEL
muestra_M = muestra_M.withColumn("label", when(col("review/score") >= 4.0, 1).otherwise(0))

# ELIMINAR NULOS EN COLUMNAS EXISTENTES
muestra_M = muestra_M.dropna(subset=["ratingsCount", "year", "categories"])

# PIPELINE DE TRANSFORMACIÓN
# ----------------------------------------
indexer = StringIndexer(inputCol="categories", outputCol="categories_index", handleInvalid="keep")
encoder = OneHotEncoder(inputCol="categories_index", outputCol="categories_encoded", handleInvalid="keep")
assembler = VectorAssembler(inputCols=["ratingsCount", "year", "categories_encoded"], outputCol="features_raw")
scaler = MinMaxScaler(inputCol="features_raw", outputCol="features")

pipeline = Pipeline(stages=[indexer, encoder, assembler, scaler])
model_prep = pipeline.fit(muestra_M)
muestra_final = model_prep.transform(muestra_M)


## 4 Preparación del conjunto de entrenamiento y Prueba

**Para el modelo supervisado (Decision Tree)**, Dividimos la muestra de datos en un conjunto de entrenamiento (train) y otro de prueba (test). Esta separación nos permite luego evaluar qué tan bien generaliza el modelo a datos no vistos.


**Para el modelo no supervisado (KMeans)**, No necesitamos una división train/test de la misma forma, ya que el objetivo no es predecir etiquetas conocidas sino agrupar todo el conjunto. Así que tomamos el conjunto completo de datos procesados (muestra_final) .En el código final, por ejemplo, probamos con **K=15** para formar diez grupos de libros, pues son muchas las categorias que existen.


In [4]:

# ----------------------------------------
# PARTE 4: Preparación del conjunto de entrenamiento y prueba
# DIVISIÓN TRAIN/TEST
train_data, test_data = muestra_final.randomSplit([0.8, 0.2], seed=42)

# CONVERSIÓN DE LABEL A DOUBLE
train_data = train_data.withColumn("label", col("label").cast("double"))
test_data = test_data.withColumn("label", col("label").cast("double"))


## 5. Construcción de modelos de aprendizaje supervisado y no supervisado

In [5]:

# ----------------------------------------
# PARTE 5: Construcción de modelos de aprendizaje supervisado y no supervisado
# MODELO SUPERVISADO
# ----------------------------------------
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")
dt_model = dt.fit(train_data)
predictions = dt_model.transform(test_data)

evaluator_acc = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")

print("Accuracy:", evaluator_acc.evaluate(predictions))
print("F1-score:", evaluator_f1.evaluate(predictions))

Accuracy: 0.7153083700440529
F1-score: 0.7111588248537959


In [6]:

# MODELO NO SUPERVISADO
# ----------------------------------------
kmeans = KMeans(featuresCol="features", k=15, seed=42)
kmeans_model = kmeans.fit(muestra_final)
clusters = kmeans_model.transform(muestra_final)

evaluator_cluster = ClusteringEvaluator(featuresCol="features", metricName="silhouette", distanceMeasure="squaredEuclidean")
print("Silhouette score:", evaluator_cluster.evaluate(clusters))
clusters.groupBy("prediction").count().show()


Silhouette score: 0.7170509886949381
+----------+-----+
|prediction|count|
+----------+-----+
|        12|  270|
|         1|  355|
|        13|  662|
|         6| 8037|
|         5|  398|
|         9|  762|
|         4| 5902|
|         8|  183|
|        10|  737|
|        11|  423|
|        14|  727|
|         0| 2487|
|         7|  519|
|         2| 4892|
|         3| 1380|
+----------+-----+



**Conclusión**

Podemos comparar el desempeño y utilidad de cada modelo en el contexto de las reseñas de libros de Amazon con Spark:

DecisionTreeClassifier (Supervisado): El árbol de decisión logró una precisión alrededor del 71% (accuracy ≈ 0.715) al clasificar las reseñas en positivas o negativas, junto con un F1-score aproximado de 0.711 en el conjunto de prueba.


KMeans (No supervisado): Al agrupar los libros mediante KMeans, en la configuración final usamos K = 15 clústeres, obteniendo un índice de silueta alrededor de 0.71. Un Silhouette score de 0.71 indica una cohesión de clúster moderadamente alta: los libros en general están bien agrupados con otros similares, aunque podría haber algo de solapamiento entre ciertos grupos.

Pero, hablando sobre Spark y las particiones. al generar una muestra representativa con lo que se analizó previamente ayudo bastante a reducir la carga que llegaría a tener un modelo implementado con otras librerias como SKlearn. Los tiempos para procesar casi 3 GB de data fueron excelentes.